In [50]:
import json
import pandas as pd
from collections import defaultdict
import requests
import shapely
from shapely.geometry import Point
from shapely.geometry import Polygon

In [71]:
#read json file
filename = "PM.json"
output = 'lga_site.json'
with open(filename, "r") as file:
    pm = json.load(file)

In [5]:
url = "https://gateway.api.epa.vic.gov.au/environmentMonitoring/v1/sites?environmentalSegment=air"

headers = {
    "User-Agent": 'curl/8.4.0',
    "Cache-Control": 'no-cache',
    "X-API-Key": "62eb03b279a54273ace3f893e994f90d"
}

In [6]:
try:
    response = requests.get(url, headers=headers)
    raw_epa = response.json()
except Exception as e:
    print("error when request EPA data: ", e)

In [33]:
site = defaultdict()
for record in raw_epa['records']:
    # site[record['siteID']] = record['geometry']['coordinates']
    site[record['siteName']] = record['geometry']['coordinates']

In [44]:
with open('VIC_LGA_2021.geojson', "r") as geo_file:
    geo = json.load(geo_file)['features']

In [45]:
geo

[{'type': 'Feature',
  'geometry': {'coordinates': [[[145.23958917900006, -38.07766384799992],
     [145.245614862, -38.04683985799994],
     [145.24723086300003, -38.03639686799994],
     [145.24683100400011, -38.02531691799993],
     [145.251859426, -38.00362220799997],
     [145.22732986000003, -38.00075685699994],
     [145.22864288100004, -37.993808866999984],
     [145.22726205100003, -37.99263770699997],
     [145.23196186100006, -37.984630866999964],
     [145.23246187100003, -37.98087786699995],
     [145.23493288100008, -37.97836585699997],
     [145.2353618720001, -37.97460686699998],
     [145.23286786300002, -37.970311866999964],
     [145.23362188300007, -37.96482485699994],
     [145.23241887400002, -37.96133784699998],
     [145.2340408640001, -37.95878686699996],
     [145.23218688500003, -37.95256385699997],
     [145.22919987500006, -37.95206285699993],
     [145.2257998660001, -37.949241865999966],
     [145.219163868, -37.94013484599998],
     [145.19250062300011, 

In [67]:
lga_site = defaultdict()
n = 0
for record in geo:
    name = record['properties']['lga_name'][0]
    print('LGA: '+name)
    shape = shapely.from_geojson(json.dumps(record['geometry']))
    for i in site.keys():
        print(i)
        inArea = shape.contains(Point(site[i][1],site[i][0]))
        if inArea:
            n += 1
            if name in lga_site.keys():
                lga_site[name].append(i)
            else: lga_site[name] = [i]
            print('matched')

LGA: Greater Dandenong
Box Hill
Alphington
Churchill
Traralgon South
Traralgon
Tyers North
Dandenong
matched
Moe
Boolarra South
Traralgon East
Boolarra
Yinnar
Morwell East
Geelong South
Willow Grove
Yallourn North
Point Cook
Flynns Creek
Melbourne CBD
Tyers
Callignee
Hernes Oak
Wangaratta
Morwell South
Brighton
Echuca
Healesville
Warrnambool
Mildura
Horsham
Altona North
Ballarat
Bairnsdale
Castlemaine
Beechworth
Benalla
Ararat
Cobden
Camperdown
Colac
Maffra
Gisborne
Macedon
Lancefield
Kyneton
Heathcote
Kinglake
Daylesford
Heywood
Hamilton
Kerang
Lorne
Mallacoota
Shepparton
Orbost
Sale
Myrtleford
Mansfield
Portland
Ouyen
Torquay
Wonthaggi
Warragul
Yarrawonga
Wodonga
Broadford
Yarra Glen
Sunbury
Lakes Entrance
Leongatha
Bacchus Marsh
Warburton
Drysdale
Rutherglen
Bright
Macleod
Swan Hill
Spotswood
Wallan
Rosedale
Kingsville
Flynn
Glengarry
Melton
Mooroolbark
Footscray
Brooklyn
Bendigo
Porepunkah
LGA: Hepburn
Box Hill
Alphington
Churchill
Traralgon South
Traralgon
Tyers North
Dandenong
Mo

In [68]:
n

89

In [66]:
lga_site

defaultdict(None,
            {'Greater Dandenong': ['Dandenong'],
             'Hepburn': ['Daylesford'],
             'Mitchell': ['Broadford', 'Wallan'],
             'Colac Otway': ['Colac'],
             'East Gippsland': ['Bairnsdale',
              'Mallacoota',
              'Orbost',
              'Lakes Entrance'],
             'Bayside': ['Brighton'],
             'Greater Geelong': ['Geelong South', 'Drysdale'],
             'Melton': ['Melton'],
             'Gannawarra': ['Kerang'],
             'Indigo': ['Beechworth', 'Rutherglen'],
             'Swan Hill': ['Swan Hill'],
             'Baw Baw': ['Willow Grove', 'Warragul'],
             'Southern Grampians': ['Hamilton'],
             'Yarra Ranges': ['Healesville',
              'Yarra Glen',
              'Warburton',
              'Mooroolbark'],
             'South Gippsland': ['Leongatha'],
             'Melbourne': ['Melbourne CBD'],
             'Moira': ['Yarrawonga'],
             'Moorabool': ['Bacchus Marsh

In [65]:
len(lga_site.keys())

46

In [70]:
with open(output,'w') as out:
    json.dump(lga_site,out,indent=2)